# **Importing the neccessary Libraries**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### **Reading the dataset**

In [ ]:
df = pd.read_csv("Global Economy Indicators.csv")

In [ ]:
df

In [ ]:
df.describe(include='all')

In [ ]:
df.columns

##### **Removing Whitespaces**

There were whitespaces in all the column names. So we used **strip()** function to remove them

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.columns

### **Checking the null values**

In [ ]:
df.isnull().sum()

#### **Filling the null values**
So here the null values are filled with both **mean()** as well as **median()**.<br>
For the columns Agriculture and Changes in Inventories, we have used median because it has positive and negative thus the mean() might be skewed in one direction or can even result the mean to go to 0 value.<br>
For rest of the columns, we have used mean() as they only have positve values.

In [ ]:
df['Agriculture, hunting, forestry, fishing (ISIC A-B)'] = df['Agriculture, hunting, forestry, fishing (ISIC A-B)'].fillna(df['Agriculture, hunting, forestry, fishing (ISIC A-B)'].median())

In [ ]:
df['Changes in inventories'] = df['Changes in inventories'].fillna(df['Changes in inventories'].median())

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
df.isnull().sum()

#### **Data Exploration**
> **Histogram** : This plot shows us the distribution of the GNI. One important thing to nte here is that we had to use log_scale as the data was heavily left-skewed.


In [ ]:
# As the real world data is heavily skewed hence I have added bins in the log scale
plt.figure(figsize=(13, 6))
sns.histplot(df['Gross National Income(GNI) in USD'],log_scale=True, kde=True)
plt.title('Histogram of Per capita GNI')
plt.xlabel('Per capita GNI')
plt.ylabel('Count')
plt.show()

> Box Plot : Here we have done two plots, one with Log of GDP and second with Log of GDP / Population.


After doing the first plot we saw two main things:<br>
1) There are many outliers.<br>
2) The GDP of some countries(for eg. USA) has always been above the average from the start. Hence we made the conclusion that the GDP should looked at by dividing it by Population.<br> Hence the countries with huge GDP do also have large Population, thus the GDP / Population plot gives us much better results and also eliminates outliers.

In [ ]:
df['Log_GDP'] = np.log1p(df['Gross Domestic Product (GDP)'])
plt.figure(figsize=(20, 6))
sns.boxplot(x=df['Log_GDP'])
plt.title('Box Plot of Log Transformed GDP')
plt.xlabel('Log Transformed GDP')
plt.show()

In [ ]:
df['GDP by Population'] = df['Gross Domestic Product (GDP)'] / df['Population']

In [ ]:
df['Log_GDP_Population'] = np.log1p(df['GDP by Population'])
plt.figure(figsize=(20, 6))
sns.boxplot(x=df['Log_GDP_Population'])
plt.title('Box Plot of Log Transformed GDP by Population')
plt.xlabel('Log Transformed GDP by Population')
plt.show()

##### **Detecting Outliers**

Here the outliers are calculated using IQR(Inter Quantile Range) for both Log GDP and Log GDP by Population. Further bar chart is plotted for better readability.

In [ ]:
Q1 = df['Log_GDP'].quantile(0.25)
Q3 = df['Log_GDP'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Log_GDP'] < lower_bound) | (df['Log_GDP'] > upper_bound)]

In [ ]:
outliers_by_country = outliers.groupby('Country').size().reset_index(name='Outlier_Count')

In [ ]:
plt.figure(figsize=(15, 6))
sns.barplot(x='Country', y='Outlier_Count', data=outliers_by_country)
plt.title('Outlier Counts by Country')
plt.xlabel('Country')
plt.ylabel('Outlier Count')
plt.xticks(rotation=90)
plt.show()

In [ ]:
Q1 = df['Log_GDP_Population'].quantile(0.25)
Q3 = df['Log_GDP_Population'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers1 = df[(df['Log_GDP_Population'] < lower_bound) | (df['Log_GDP_Population'] > upper_bound)]

In [ ]:
outliers_by_country = outliers1.groupby('Country').size().reset_index(name='Outlier_Count')

print(outliers_by_country)

In [ ]:
print(df['Population'].describe())

In [ ]:
df.dtypes

In [ ]:
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

> Correlation Matrix : Plotting the correlation matrix using heatmap tells us whether two features are correlated or not.

In [ ]:
correlation_matrix = df[numerical_columns].corr()
plt.figure(figsize=(16, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

GNI and GDP have a nearly perfect positive correlation with each other.

In [ ]:
numerical_feature1 = 'Gross National Income(GNI) in USD'
numerical_feature2 = 'Gross Domestic Product (GDP)'

plt.figure(figsize=(16, 12))
sns.scatterplot(x=df[numerical_feature1], y=df[numerical_feature2], palette='viridis', alpha=0.7)
plt.title(f'Scatter Plot between {numerical_feature1} and {numerical_feature2}')
plt.xlabel(numerical_feature1)
plt.ylabel(numerical_feature2)
plt.show()

In [ ]:
sns.pairplot(data=df, vars=['Gross National Income(GNI) in USD','Population','Gross Domestic Product (GDP)'], hue='Year',palette="plasma")
plt.figure(figsize=(16,12))
plt.show()

> **Line Plot :** Showing the GDP by Population trend for the top 10 countries by GDP.<br> Note : Russia's data has been split into two parts - USSR(1970-90) and Russian Federation(1990-2021)

In [ ]:
top_countries_avg_gdp = df.groupby('Country')['Gross Domestic Product (GDP)'].mean().nlargest(10).index

df_top10 = df[df['Country'].isin(top_countries_avg_gdp)]

plt.figure(figsize=(12, 6))
sns.lineplot(x='Year', y='Log_GDP_Population', hue='Country', palette='gist_rainbow', data=df_top10)
plt.title('GDP by Population Trend Over Time for Top 10 Countries')
plt.xlabel('Year')
plt.ylabel('GDP')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
top_countries_avg_gdp

In [ ]:
df['Gross Domestic Product (GDP) in Million USD'] = df['Gross Domestic Product (GDP)']/10**6
pc_gni_g16 = pd.pivot_table(df[df["Country"].isin([' Argentina ',' Australia ',' Brazil ',' Canada ',
                                                         ' China ',' India ',' France ',' Italy ',' Japan ',
                                                         ' Republic of Korea ',' Mexico ',' Russia Federation ',
                                                         ' Saudi Arabia ',' Türkiye ',' United Kingdom ', ' United States '])],
                            values='Gross Domestic Product (GDP) in Million USD',
                            index=['Year'],
                           columns=['Country'],
                            aggfunc='sum',
                            fill_value=0)

zero_sum_cols = pc_gni_g16.columns[pc_gni_g16.sum() == 0]

pc_gni_g16 = pc_gni_g16.drop(zero_sum_cols, axis=1)
pc_gni_g16.shape

In [ ]:
pip install bar_chart_race

> **Bar Chart Race : A unique plot to show how the Country's GDP trend was for all the years.**

In [ ]:
import bar_chart_race as bcr

bcr.bar_chart_race(df = pc_gni_g16,
                   n_bars = 16,
                   sort='desc',
                   title='Gross Domestic Product (GDP) in Million USD',
                   dpi=100,
                   steps_per_period=1,
                   period_length=1000)

### **Regression Analysis**
> Firstly selecting only a few features having correlation with the target variable GDP by Population.

In [ ]:
features = ['Final consumption expenditure',
 'General government final consumption expenditure',
 'Household consumption expenditure (including Non-profit institutions serving households)',
 'Other Activities (ISIC J-P)',
 'Transport, storage and communication (ISIC I)',
 'Wholesale, retail trade, restaurants and hotels (ISIC G-H)',
 'Gross National Income(GNI) in USD',
 'Total Value Added']

target_variable = df['Log_GDP_Population']

In [ ]:
X = df[features]
y = target_variable

> Scaling all the features into a certain range.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

> Using Shuffle Split to eliminate further bias.

In [ ]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

for train_index, test_index in shuffle_split.split(X, y):
    X_train, X_test = X_normalized[train_index], X_normalized[test_index]
    y_train, y_test = y[train_index], y[test_index]

##### Using Linear Regression, Support Vector Regressor and Random Forest Regressor for predicting the GDP.

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)

In [ ]:
from sklearn.svm import SVR

svr_model = SVR(kernel="rbf")
svr_model.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

random_forest_model = RandomForestRegressor()
random_forest_model.fit(X_train, y_train)

###### **Evaluation Metrics** : Used RMSE and R2 score to evaluate all the algorithms.
> Outoff all the algorithms, **Random Forest** performs the best giving the lowest RMSE and the highest R2 (goodness of fit).

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

linear_regression_predictions = linear_regression_model.predict(X_test)
linear_regression_rmse = np.sqrt(mean_squared_error(y_test, linear_regression_predictions))
linear_regression_r2 = r2_score(y_test, linear_regression_predictions)

print("Linear Regression RMSE:", linear_regression_rmse)
print("Linear Regression R-squared:", linear_regression_r2)

svr_predictions = svr_model.predict(X_test)
svr_rmse = np.sqrt(mean_squared_error(y_test, svr_predictions))
svr_r2 = r2_score(y_test, svr_predictions)

print("Support Vector Regression RMSE:", svr_rmse)
print("Support Vector Regression R-squared:", svr_r2)

random_forest_predictions = random_forest_model.predict(X_test)
random_forest_rmse = np.sqrt(mean_squared_error(y_test, random_forest_predictions))
random_forest_r2 = r2_score(y_test, random_forest_predictions)

print("Random Forest Regression RMSE:", random_forest_rmse)
print("Random Forest Regression R-squared:", random_forest_r2)

#### Time Series Analysis <br>
Used SARIMA (Seasonal AutoRegressive Integrated Moving Average) specifcally because our data is seasonal data, and the ARIMA model makes an assumption that the data is not seasonal thus performing worse.

> We have used the Gross Domestic Product (GDP) feature and scaled it manually by dividing it by 10^9 to convert into Billions.

Lastly, we have only shown the prediction for Top 10 countries because our dataset has 220 countries and showing it will take lot of computation time.<br>
The prediction starts from the year 2020 and goes upto 2060.

In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

df['Year'] = pd.to_datetime(df['Year'], format='%Y')

df['Gross Domestic Product (GDP) Scaled'] = df['Gross Domestic Product (GDP)'] / 10**9
avg_gdp_by_country = df.groupby('Country')['Gross Domestic Product (GDP) Scaled'].mean()

top_10_countries = avg_gdp_by_country.nlargest(10).index

for country in top_10_countries:

    df_country = df[df['Country'] == country]

    df_country = df_country.set_index('Year')

    future_years = pd.date_range(start='2021-01-01', end='2060-01-01', freq='A')
    future_index = df_country.index.union(future_years)

    train_size = int(len(df_country) * 1)
    train, test = df_country.iloc[:train_size], df_country.iloc[train_size:]

    model = SARIMAX(train['GDP by Population'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)) #Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors model
    fit_model = model.fit()

    """p: Trend autoregression order.(relationship between previous and current value)
    d: Trend difference order.(stationary)
    q: Trend moving average order.(current value and the residual error)"""

    forecast = fit_model.get_forecast(steps=len(future_years))
    future_predictions = forecast.predicted_mean

    plt.plot(df_country.index, df_country['GDP by Population'], label='Actual')
    plt.plot(future_predictions.index, future_predictions, label='Predicted (Future)')
    plt.title(f'{country} - GDP Forecasting')
    plt.legend()
    plt.show()

    gdp_2021 = df_country.loc['2021-01-01', 'Gross Domestic Product (GDP) Scaled']
    predicted_gdp_last_year = future_predictions.index[-1]
    predicted_gdp_last_year_value = future_predictions.iloc[-1]
    print(f"{country} - GDP in billions in 2021: {gdp_2021:.2f}")
    print(f"{country} - Predicted GDP in billions in {predicted_gdp_last_year.year}: {predicted_gdp_last_year_value:.2f}")
    print("="*40)